# Importing Data from EOD Data

In [ ]:
token = "INSERT YOUR PERSONAL TOKEN HERE!"

## Introduction to the API (hands-on)

In [ ]:
import pandas as pd

In [ ]:
url = "https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX&period=d"

In [ ]:
pd.read_csv(url)

## Getting Historical Stock Prices and Volume Data

In [ ]:
import pandas as pd

In [ ]:
def stock_prices(ticker, start, end, freq):
    url = "https://eodhistoricaldata.com/api/eod/{}?from={}&to={}&api_token={}&period={}".format(ticker, start, end, token, freq)
    df = pd.read_csv(url, index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
stock_prices(ticker = "MSFT", start = "1900-01-01", end = "2019-08-24", freq = "m")

In [ ]:
stock_prices("MSFT", start = "1900-01-01", end = "2019-08-24", freq = "w")

In [ ]:
msft = stock_prices("MSFT", start = "1900-01-01", end = "2019-08-24", freq = "d")

In [ ]:
msft.tail()

In [ ]:
import yfinance as yf

In [ ]:
yf.download("MSFT").tail()

## Stock Splits and Dividends

In [ ]:
import pandas as pd

In [ ]:
def dividends(ticker, start):
    url = "https://eodhistoricaldata.com/api/div/{}?api_token={}&from={}".format(ticker, token, start)
    df = pd.read_csv(url, index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
dividends("MSFT","1900-01-01")

In [ ]:
def splits(ticker, start):
    url = "https://eodhistoricaldata.com/api/splits/{}?api_token={}&from={}".format(ticker, token, start)
    df = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]["Stock Splits"]
    if len(df) == 0:
        df = pd.DataFrame(columns = ["Splits"])
        df.index.name = "Date"
    else:
        df = df.str.split("/", expand = True).astype("float")
        df.index = pd.to_datetime(df.index)
        df = df.iloc[:, 0].div(df.iloc[:,1]).to_frame()
        df.columns = ["Splits"]
    return df

In [ ]:
splits("MSFT", "1900-01-01")

In [ ]:
def stock_prices_actions(ticker, start, end, freq):
    
    url = "https://eodhistoricaldata.com/api/eod/{}?from={}&to={}&api_token={}&period={}".format(ticker, start, end, token, freq)
    df = pd.read_csv(url, index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    
    url = "https://eodhistoricaldata.com/api/div/{}?api_token={}&from={}".format(ticker, token, start)
    df_d = pd.read_csv(url, index_col = "Date").iloc[:-1]
    df_d.index = pd.to_datetime(df_d.index)
    if len(df_d) == 0:
        df_d = pd.DataFrame([0.0], index = pd.to_datetime([start]), columns = ["Dividends"])
    
    url = "https://eodhistoricaldata.com/api/splits/{}?api_token={}&from={}".format(ticker, token, start)
    df_s = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]["Stock Splits"]
    if len(df_s) == 0:
        df_s = pd.DataFrame([1.0], index = pd.to_datetime([start]), columns = ["Splits"])
    else:
        df_s = df_s.str.split("/", expand = True).astype("float")
        df_s.index = pd.to_datetime(df_s.index)
        df_s = df_s.iloc[:, 0].div(df_s.iloc[:,1]).to_frame()
        df_s.columns = ["Splits"]
    
    df["Dividends"] = df_d
    df.Dividends.fillna(0, inplace = True)
    df["Splits"] = df_s
    df.Splits.fillna(1, inplace = True)
    
    return df

In [ ]:
stock_prices_actions("MSFT", "1900-01-01", "2019-08-24", "d")

In [ ]:
msft = stock_prices_actions("MSFT", "1900-01-01", "2019-08-24", "d")

In [ ]:
msft.head()

In [ ]:
msft.tail()

In [ ]:
import yfinance as yf

In [ ]:
ya = yf.download("MSFT", end = "2019-08-24", actions = True)

In [ ]:
ya.tail()

In [ ]:
ya[ya["Stock Splits"] > 0]

In [ ]:
msft[msft.Splits > 1] 

In [ ]:
ya.loc["2003-02-13":"2003-02-22"]

In [ ]:
msft.loc["2003-02-13":"2003-02-22"]

In [ ]:
ya[ya["Dividends"] > 0].tail()

In [ ]:
msft[msft.Dividends > 0].tail()

## Financial Indexes

In [ ]:
import pandas as pd

In [ ]:
def index_prices(ticker, start, end, freq):
    url = "https://eodhistoricaldata.com/api/eod/{}.INDX?from={}&to={}&api_token={}&period={}".format(ticker, start, end, token, freq)
    df = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
ticker = "GSPC"
ticker2 = "DJI"
ticker3 = "SP500TR"

In [ ]:
index_prices(ticker, start = "1900-01-01", end = "2019-08-24", freq = "d")

In [ ]:
index_prices(ticker2, start = "2010-01-01", end = "2019-08-24", freq = "w")

In [ ]:
index_prices(ticker3, start = "1900-01-01", end = "2019-08-24", freq = "d")

## Currencies / FX

In [ ]:
import pandas as pd

In [ ]:
def fx_rates(pair, start, end, freq):
    url = "https://eodhistoricaldata.com/api/eod/{}.FOREX?from={}&to={}&api_token={}&period={}".format(pair, start, end, token, freq)
    df = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
fx_rates("EURUSD", start = "1900-01-01", end = "2019-08-24", freq = "d")

In [ ]:
fx_rates("EUR", start = "1900-01-01", end = "2019-08-24", freq = "d")

In [ ]:
fx_rates("USDGBP", start = "1900-01-01", end = "2019-08-24", freq = "d")

## Cryptocurrencies

In [ ]:
import pandas as pd

In [ ]:
def cc_rates(symbol, start, end, freq):
    url = "https://eodhistoricaldata.com/api/eod/{}-USD.CC?from={}&to={}&api_token={}&period={}".format(symbol, start, end, token, freq)
    df = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
cc_rates("BTC", "2000-01-01", end = "2019-08-24", freq = "d")

In [ ]:
cc_rates("ETH", "2010-01-01", end = "2019-08-24", freq = "w")

In [ ]:
cc =cc_rates("ETH", "2010-01-01", end = "2019-08-24", freq = "w")

In [ ]:
cc.head()

## Commodities

In [ ]:
import pandas as pd

In [ ]:
def commodity_prices(symbol, start, end, freq):
    url = "https://eodhistoricaldata.com/api/eod/{}.COMM?from={}&to={}&api_token={}&period={}".format(symbol, start, end, token, freq)
    df = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
commodity_prices("GC", start = "1900-01-01", end = "2019-08-24", freq = "d")

In [ ]:
commodity_prices("CL", start = "1900-01-01", end = "2019-08-24", freq = "m")

## Mutual Funds & ETFs

In [ ]:
import pandas as pd

In [ ]:
ticker = "TLT" # iShares 20+ Year Treasury Bond ETF
ticker2 = "OMOIX" # Vivaldi Multi-Strategy Fund Class I

In [ ]:
def fund_prices_actions(ticker, start, end, freq):
    
    url = "https://eodhistoricaldata.com/api/eod/{}?from={}&to={}&api_token={}&period={}".format(ticker, start, end, token, freq)
    df = pd.read_csv(url, index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    
    url = "https://eodhistoricaldata.com/api/div/{}?api_token={}&from={}".format(ticker, token, start)
    df_d = pd.read_csv(url, index_col = "Date").iloc[:-1]
    df_d.index = pd.to_datetime(df_d.index)
    if len(df_d) == 0:
        df_d = pd.DataFrame([0.0], index = pd.to_datetime([start]), columns = ["Dividends"])
    
    url = "https://eodhistoricaldata.com/api/splits/{}?api_token={}&from={}".format(ticker, token, start)
    df_s = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]["Stock Splits"]
    if len(df_s) == 0:
        df_s = pd.DataFrame([1.0], index = pd.to_datetime([start]), columns = ["Splits"])
    else:
        df_s = df_s.str.split("/", expand = True).astype("float")
        df_s.index = pd.to_datetime(df_s.index)
        df_s = df_s.iloc[:, 0].div(df_s.iloc[:,1]).to_frame()
        df_s.columns = ["Splits"]
    
    df["Dividends"] = df_d
    df.Dividends.fillna(0, inplace = True)
    df["Splits"] = df_s
    df.Splits.fillna(1, inplace = True)
    
    return df

In [ ]:
fund_prices_actions(ticker, start = "1900-01-01", end = "2019-08-24", freq = "d")

In [ ]:
fund_prices_actions(ticker2, start = "1900-01-01", end = "2019-08-24", freq = "d")

## Treasury Yields

In [ ]:
import pandas as pd

In [ ]:
ticker1 = "TNX" # 10Y Treasury Yield
ticker2 = "FVX" # 5Y Treasury Yield

In [ ]:
def treasury_yields(ticker, start, end, freq):
    url = "https://eodhistoricaldata.com/api/eod/{}.INDX?from={}&to={}&api_token={}&period={}".format(ticker, start, end, token, freq)
    df = pd.read_csv(url, parse_dates = ["Date"], index_col = "Date").iloc[:-1]
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
treasury_yields("TNX", start = "1900-01-01", end = "2019-08-24", freq = "d")

In [ ]:
treasury_yields(ticker2, start = "1900-01-01", end = "2019-08-24", freq = "d")

## Stock Fundamentals, Meta Info and Performance Metrics

In [ ]:
import pandas as pd
import requests

In [ ]:
def meta_general(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["General"])

In [ ]:
meta_general("MSFT")

In [ ]:
def meta_highl(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["Highlights"])

In [ ]:
meta_highl("MSFT")

In [ ]:
def meta_value(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["Valuation"])

In [ ]:
meta_value("MSFT")

In [ ]:
def meta_share(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["SharesStats"])

In [ ]:
meta_share("MSFT")

In [ ]:
def meta_tech(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["Technicals"])

In [ ]:
meta_tech("MSFT")

In [ ]:
def meta_spldiv(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()['SplitsDividends'])

In [ ]:
meta_spldiv("MSFT")

In [ ]:
def meta_outshares(ticker):
    return pd.DataFrame(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["outstandingShares"]["quarterly"]).T

In [ ]:
meta_outshares("MSFT")

In [ ]:
def meta_earn(ticker):
    return pd.DataFrame(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()['Earnings']["History"]).T

In [ ]:
meta_earn("MSFT")

## Financials (Balance Sheet, Cashflows, P&L)

In [ ]:
import pandas as pd
import requests

In [ ]:
def balance(ticker):
    return pd.DataFrame(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["Financials"]["Balance_Sheet"]["yearly"])

In [ ]:
balance("MSFT")

In [ ]:
def cashflow(ticker):
    return pd.DataFrame(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["Financials"]["Cash_Flow"]["yearly"])

In [ ]:
cashflow("MSFT")

In [ ]:
def income(ticker):
    return pd.DataFrame(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["Financials"]["Income_Statement"]["yearly"])

In [ ]:
income("MSFT")

## Fundamentals for Funds & ETFs 

In [ ]:
import pandas as pd
import requests

In [ ]:
ticker = "TLT" # iShares 20+ Year Treasury Bond ETF
ticker2 = "OMOIX" # Vivaldi Multi-Strategy Fund Class I

In [ ]:
def etf_general(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["General"])

In [ ]:
etf_general(ticker)

In [ ]:
etf_general(ticker).Description

In [ ]:
def etf_technicals(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["Technicals"])

In [ ]:
etf_technicals(ticker)

In [ ]:
def etf_breakdowns(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["ETF_Data"])

In [ ]:
etf_breakdowns(ticker)

In [ ]:
def fund_general(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["General"])

In [ ]:
fund_general(ticker2)

In [ ]:
def fund_data(ticker):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}".format(ticker, token)).json()["MutualFund_Data"])

In [ ]:
fund_data(ticker2)

## Bond Data: Fundamentals

In [ ]:
import pandas as pd
import requests

In [ ]:
ident = "910047AG4"

In [ ]:
def bond_meta(ident):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/bond-fundamentals/{}?api_token={}".format(ident, token)).json())

In [ ]:
bond_meta(ident)

In [ ]:
bond_meta(ident).IssueData

In [ ]:
def bond_class(ident):
    return pd.Series(requests.get("https://eodhistoricaldata.com/api/bond-fundamentals/{}?api_token={}".format(ident, token)).json()["ClassificationData"])

In [ ]:
bond_class(ident)

## Bond Data: Ratings

In [ ]:
import pandas as pd
import requests

In [ ]:
ident = "910047AG4"

In [ ]:
def bond_rating(ident):
    rating = pd.Series(requests.get("https://eodhistoricaldata.com/api/bond-fundamentals/{}?api_token={}".format(ident, token)).json()["Rating"])
    return pd.DataFrame(index = rating[~rating.index.str.contains("Update")].index, data = {"Rating": rating[~rating.index.str.contains("Update")].values, "Update_Date": rating[rating.index.str.contains("Update")].values})

In [ ]:
bond_rating(ident)

## Bond Data: Historical Prices, Yields and Volume

In [ ]:
import pandas as pd
import requests

In [ ]:
Isin = "US910047AG49"

In [ ]:
def bond_hist(Isin, start):
    return pd.read_csv("https://eodhistoricaldata.com/api/eod/{}.BOND?api_token={}&fmt=csv&from={}".format(Isin, token, start), parse_dates = ["Date"], index_col = "Date").iloc[:-1]

In [ ]:
bond_hist(Isin, "2013-11-08")

## All Ticker Symbols for entire Exchanges

In [ ]:
import pandas as pd

In [ ]:
def ex_symbols(exchange):
    return pd.read_csv("https://eodhistoricaldata.com/api/exchanges/{}?api_token={}&fmt=csv".format(exchange, token)).iloc[:-1]

In [ ]:
ex_symbols("US")

In [ ]:
us = ex_symbols("US")

In [ ]:
us.Type.value_counts()

In [ ]:
ex_symbols("MU")

In [ ]:
ex_symbols("INDX")

In [ ]:
ex_symbols("COMM")

In [ ]:
ex_symbols("CC")

## Bulk Download of Prices, Splits & Dividends for entire Exchanges 

In [ ]:
import pandas as pd

In [ ]:
def bulk_price(exchange, date):
    return pd.read_csv("http://eodhistoricaldata.com/api/eod-bulk-last-day/{}?api_token={}&fmt=csv&date={}".format(exchange, token, date)).iloc[:-1]

In [ ]:
bulk_price("US", "2018-05-04")

In [ ]:
bulk_price("LSE", "2018-05-04")

In [ ]:
def bulk_split(exchange, date):
    return pd.read_csv("http://eodhistoricaldata.com/api/eod-bulk-last-day/{}?api_token={}&type=splits&date={}".format(exchange, token, date)).iloc[:-1]

In [ ]:
bulk_split("US", "2018-05-04")

In [ ]:
def bulk_dividend(exchange, date):
    return pd.read_csv("http://eodhistoricaldata.com/api/eod-bulk-last-day/{}?api_token={}&type=dividends&date={}".format(exchange, token, date)).iloc[:-1]

In [ ]:
bulk_dividend("LSE", "2018-05-04")